In [0]:
#### Data Preparation Cell

import numpy as np
import torch.nn
import torch.nn.functional 
import os



from google.colab import drive
drive.mount('/content/drive')

train_x      = np.load("/content/drive/My Drive/Deep_project/Train.npy"     )
test_x       = np.load("/content/drive/My Drive/Deep_project/Test.npy"      )
validation_x = np.load("/content/drive/My Drive/Deep_project/Validation.npy")


with open('/content/drive/My Drive/Deep_project/formulas/train_formulas.txt', 'r') as f:
    train_y = f.readlines()
    
with open('/content/drive/My Drive/Deep_project/formulas/validation_formulas.txt', 'r') as f:
    validation_y = f.readlines()


###############################################################

token_dict={}
token_to_index={}
index_to_token={}


for sentence in train_y:
  for token in sentence.split():
    token_dict[token]=1
  
token_list=token_dict.keys()


for index,token in enumerate(token_list,0):
  token_to_index[token]=index
  index_to_token[index]=token
  
  
bos_index=len(token_list)
eos_index=len(token_list)+1
index_to_token[len(token_list)]='bos'
index_to_token[len(token_list)+1]='eos'


##############################################################

maximum_length_formula=0
maxim = 0 

for i in range(len(train_y)):
  if len(train_y[i].split())>maximum_length_formula:
    maximum_length_formula=len(train_y[i].split())
    
for i in range(len(validation_y)):
  if len(validation_y[i].split())>maximum_length_formula:
    maximum_length_formula=len(validation_y[i].split())

    
#########################################################    
    
train_y_processed = np.ones((len(train_y),maximum_length_formula+3))*eos_index

for i in range(len(train_y)):
  
  sentence = train_y[i].split()
  train_y_processed[i,0]=bos_index
  train_y_processed[i,-1]=len(sentence)+2
  
  for j in range(len(sentence)):
    train_y_processed[i,j+1]=token_to_index[sentence[j]]




In [0]:
### Model Definition Cell




class reshape_module(torch.nn.Module):

    def __init__(self, *args):
        super(reshape_module, self).__init__()

    def forward(self, x):
        return x.transpose(dim0=1, dim1=3).contiguous().view(x.shape[0], 50, 512 * 7)


class Decoder(torch.nn.Module):
    def __init__(self, embedding_dim, decoder_hidden_dim, encoder_hidden_dim, attention_first_layer_dim, O_dim,
                 token_num, max_length):
        super(Decoder, self).__init__()

        self.decoder_hidden_dim = decoder_hidden_dim
        self.attention_first_layer_dim = attention_first_layer_dim
        self.O_dim = O_dim
        self.max_length = max_length
        self.token_num = token_num
        self.encoder_hidden_dim = encoder_hidden_dim

        self.embedding = torch.nn.Embedding(token_num, embedding_dim)

        self.W_h = torch.nn.Linear(decoder_hidden_dim, attention_first_layer_dim)
        self.W_v = torch.nn.Linear(encoder_hidden_dim, attention_first_layer_dim)
        self.beta = torch.nn.Linear(attention_first_layer_dim, 1)
        self.W_c_h = torch.nn.Linear(decoder_hidden_dim, O_dim)
        self.W_c_c = torch.nn.Linear(encoder_hidden_dim, O_dim)
        self.W_out = torch.nn.Linear(O_dim, token_num)

        self.lstm = torch.nn.LSTM(input_size=embedding_dim + O_dim, hidden_size=512, num_layers=1, bias=True,
                                  batch_first=False, bidirectional=False)

    def forward(self, y_before, o_before, c_memory_before, h_before, encoder_hiddens):
        
        input = torch.cat((self.embedding(y_before), o_before), dim=1).unsqueeze(dim=0)

        _, (h_t, c_t) = self.lstm(input, (h_before, c_memory_before))

        attention_first_layer_output = self.W_v(encoder_hiddens) + torch.unsqueeze(self.W_h(h_t.squeeze(dim=0)), 1)
        attention_e = self.beta(torch.nn.functional.tanh(attention_first_layer_output)).squeeze(dim=2)
        attention_alpha = torch.nn.functional.softmax(attention_e, dim=1)

        context = torch.sum(attention_alpha.unsqueeze(2) * encoder_hiddens, dim=1)

        o_t = self.W_c_c(context) + self.W_c_h(h_t.squeeze(dim=0))
        p_t = torch.nn.functional.log_softmax(self.W_out(o_t), dim=1)

        return p_t, o_t, c_t, h_t


class Model():

    def __init__(self, *args):

        self.init_conv_encoder()

        self.decoder = Decoder(embedding_dim=80, decoder_hidden_dim=512, encoder_hidden_dim=512,
                               attention_first_layer_dim=200, O_dim=512, token_num=564, max_length=50).cuda()

        self.optimizer = torch.optim.Adam(params=list(self.conv_encoder.parameters()) + list(self.decoder.parameters()), lr=0.001)
        
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=1, gamma=0.9)
    
    def init_conv_encoder(self):
        self.conv_encoder = torch.nn.Sequential(

            #####################################################################

            torch.nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1,
                            padding=1, dilation=1, groups=1,
                            bias=True, padding_mode='zeros'),

            torch.nn.ReLU(inplace=True),

            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            #####################################################################

            torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1,
                            padding=1, dilation=1, groups=1,
                            bias=True, padding_mode='zeros'),

            torch.nn.ReLU(inplace=True),

            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            ######################################################################

            torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1,
                            padding=1, dilation=1, groups=1,
                            bias=True, padding_mode='zeros'),

            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(inplace=True),

            ######################################################################

            torch.nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1,
                            padding=1, dilation=1, groups=1,
                            bias=True, padding_mode='zeros'),

            torch.nn.ReLU(inplace=True),

            torch.nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0),

            ######################################################################

            torch.nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1,
                            padding=1, dilation=1, groups=1,
                            bias=True, padding_mode='zeros'),

            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(inplace=True),

            ######################################################################

            torch.nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0),

            torch.nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1,
                            padding=1, dilation=1, groups=1,
                            bias=True, padding_mode='zeros'),

            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(inplace=True),

            reshape_module(),

            torch.nn.LSTM(input_size=7 * 512, hidden_size=256, num_layers=1, bias=True, batch_first=True,
                          bidirectional=True)

        )
        self.conv_encoder.cuda()


    def save_model(self,path,name_string):

        dest_path = path+"/"+name_string
        os.mkdir(dest_path)

        torch.save(self.conv_encoder.state_dict(),dest_path+"/conv_encoder.pt")
        torch.save(self.decoder.state_dict(),dest_path+"/decoder.pt")


    def load_model(self,path,name_string):

        dest_path = path + "/" + name_string

        self.conv_encoder.load_state_dict(torch.load(dest_path+"/conv_encoder.pt"))
        self.decoder.load_state_dict(torch.load(dest_path+"/decoder.pt"))

    def eval_mode(self):
        self.conv_encoder.eval()
        self.decoder.eval()

    def train_mode(self):
        self.conv_encoder.train()
        self.decoder.train()



    def teacher_force_train(self, image, y):
        encoder_hiddens, _ = self.conv_encoder(image)

        o_tmp = torch.Tensor(np.zeros((image.shape[0], self.decoder.O_dim))).cuda()
        c_memory_tmp = torch.Tensor(np.zeros((1, image.shape[0], self.decoder.decoder_hidden_dim))).cuda()
        h_tmp = encoder_hiddens[:, -1, :].unsqueeze(0)
        encoder_hiddens_tmp = encoder_hiddens
        loss = torch.Tensor(np.zeros((1))).cuda()

        for time_step in range(y.shape[1] - 1):
            # (self, y_before,o_before,c_memory_before, h_before, encoder_hiddens)
            # p_t,o_t,c_t,h_t

            p_t, o_tmp, c_memory_tmp, h_tmp = self.decoder(y_before=y[:, time_step], o_before=o_tmp,
                                                           c_memory_before=c_memory_tmp, h_before=h_tmp,
                                                           encoder_hiddens=encoder_hiddens_tmp)

            loss_tmp = torch.nn.functional.nll_loss(input=p_t, target=y[:, time_step + 1], reduction='mean')
            loss += loss_tmp

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()

    

    def test(self,image):

        encoder_hiddens, _ = self.conv_encoder(image)

        o_tmp = torch.Tensor(np.zeros((1, self.decoder.O_dim))).cuda()
        c_memory_tmp = torch.Tensor(np.zeros((1, 1, self.decoder.decoder_hidden_dim))).cuda()
        h_tmp = encoder_hiddens[:, -1, :].unsqueeze(0)
        encoder_hiddens_tmp = encoder_hiddens
        y_tmp = torch.LongTensor(np.ones(1) * bos_index).cuda()

        result_list = []
        result_string = ""

        counter = 0

        while y_tmp != eos_index and len(result_list) < 180:
            counter += 1
            p_t, o_tmp, c_memory_tmp, h_tmp = self.decoder(y_before=y_tmp, o_before=o_tmp, c_memory_before=c_memory_tmp,
                                                           h_before=h_tmp, encoder_hiddens=encoder_hiddens_tmp)

            y_tmp = p_t.argmax(dim=1)
            result_list.append(y_tmp)
            result_string += (index_to_token[y_tmp.item()] + " ")

        return result_string[0:-5]

    
    
    
    
    def make_formula_file(self,x,file_name):


      string_result=""

      for i in range(x.shape[0]):
        formula_string = self.test(torch.Tensor(x[i]/128-1).unsqueeze(0).unsqueeze(0).cuda())                                          
        string_result+=(formula_string+"\n")

        if i%500==0:
          print (i)

      open(file_name,'w').write(string_result)
      

 
    


  

In [0]:
## Training Cell


epoch_num = 30
batch_size = 128

#################

#os.mkdir("/content/drive/My Drive/Deep_project/version10_models")
#my_model = Model()
#my_model.load_model("/content/drive/My Drive/Deep_project/version3_models","epoch_25")
#my_model.optimizer = torch.optim.Adam(params=list(my_model.conv_encoder.parameters()) + list(my_model.decoder.parameters()), lr=4e-5)     
#my_model.scheduler = torch.optim.lr_scheduler.StepLR(my_model.optimizer, step_size=1, gamma=0.9)

#################


for epoch_counter in range(epoch_num):
  
  print("Epoch number : ",epoch_counter)
  
  permutation = np.arange(train_x.shape[0])
  np.random.shuffle(permutation)
    
  for batch_counter in range(int(train_x.shape[0]/batch_size)-2):
    
    batch_max_label_length = int(train_y_processed[permutation[batch_size*batch_counter:batch_size*(batch_counter+1)]][:,-1].max())
                                    
    batch_x = torch.Tensor(              train_x[permutation[batch_size*batch_counter:batch_size*(batch_counter+1)]]/128 -1).unsqueeze(1).cuda()
    batch_y = torch.LongTensor(train_y_processed[permutation[batch_size*batch_counter:batch_size*(batch_counter+1)]][:,0:batch_max_label_length+1]).cuda()
    
    loss = my_model.teacher_force_train(batch_x,batch_y)
    
    if batch_counter%50 == 0:
      print("batch_number= ",batch_counter," , loss= ",loss)
      
  
  my_model.save_model("/content/drive/My Drive/Deep_project/version10_models","epoch_"+str(epoch_counter))
  my_model.scheduler.step()
    







Epoch number :  0


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


batch_number=  0  , loss=  717.60498046875
batch_number=  50  , loss=  165.49234008789062
batch_number=  100  , loss=  148.3910369873047
batch_number=  150  , loss=  144.10113525390625
batch_number=  200  , loss=  134.6372528076172
batch_number=  250  , loss=  128.30055236816406
batch_number=  300  , loss=  123.25305938720703
batch_number=  350  , loss=  115.32899475097656
batch_number=  400  , loss=  112.43504333496094
batch_number=  450  , loss=  116.22109985351562
batch_number=  500  , loss=  107.33399963378906
Epoch number :  1
batch_number=  0  , loss=  112.60838317871094
batch_number=  50  , loss=  107.84037780761719
batch_number=  100  , loss=  106.7607192993164
batch_number=  150  , loss=  99.85017395019531
batch_number=  200  , loss=  97.67574310302734
batch_number=  250  , loss=  94.60527038574219
batch_number=  300  , loss=  96.83824157714844
batch_number=  350  , loss=  84.03682708740234
batch_number=  400  , loss=  86.33174133300781
batch_number=  450  , loss=  76.69016265

In [0]:
###Result text file generation cell 

validation_model = Model()
validation_model.load_model(                    "/content/drive/My Drive/Deep_project/version10_models","epoch_29")
validation_model.eval_mode()
validation_model.make_formula_file(train_x,     "/content/drive/My Drive/Deep_project/version10_models/train_pred_v10_29.txt")


# python3 Evaluation/bleu_score.py --target-formulas Dataset/formulas/validation_formulas.txt --predicted-formulas danial_validation_v3_24.txt --ngram 5
# python3 Evaluation/edit_distance.py --target-formulas Dataset/formulas/validation_formulas.txt --predicted-formulas danial_validation_v3_25.txt

    
  
  
  
  
      

In [0]:
###Chelknevis Cell

import pickle
f = open("/content/drive/My Drive/Deep_project/version10_models/optimizer_29", "wb")

pickle.dump(my_model.optimizer, f)
f.close()